In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tifffile as tiff
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd


In [ ]:
def get_image_id(data): 

    if data == "train":
        train = list({file[:9] for file in os.listdir('/kaggle/input/hubmap-kidney-segmentation/train')})
        return train
    
    elif data == "test":
        test = list({file[:9] for file in os.listdir('/kaggle/input/hubmap-kidney-segmentation/test')})
        return test  
    

In [ ]:
print("Training data sets \n")
train_ids = get_image_id("train")
print(train_ids)

print("\n Testing data sets \n")
test_ids = get_image_id("test")

print(test_ids)

## Dimensions of all tiff images

In [ ]:
# print("Training data sets \n")

# for id_im in train_ids:
#     image = tiff.imread('/kaggle/input/hubmap-kidney-segmentation/train/{}.tiff'.format(id_im))
#     print(str(image.shape) + '  ' + id_im)
    
# print("Testing data set \n")
# for id_im in test_ids:
#     image = tiff.imread('/kaggle/input/hubmap-kidney-segmentation/test/{}.tiff'.format(id_im))
#     print(str(image.shape) + ' ' + id_im)

## Generate mask

In [ ]:
def generate_mask(file_name, inv):
    train_ids = ['2f6ecfcdf', 'cb2d976f4', '1e2425f28', '54f2eec69', '0486052bb', 'aaa6a05cc', 'e79de561c', '095bf7a1f']
    test_ids = ['afa5e8098', 'c68fe75ea', 'b2dc8411c', 'b9a3865fc', '26dc41664']
    
    if file_name in train_ids:
        train_or_test = "train"
    elif file_name in test_ids:
        train_or_test = "test"
    else:
        print("ID not available in dataset")
        return
    
    image = tiff.imread('/kaggle/input/hubmap-kidney-segmentation/{}/{}.tiff'.format(train_or_test, file_name))
    dim = image.ndim
    
    if dim == 5:
        rgb_img = image[0, 0, :, :, :].transpose((1, 2, 0))
        
    elif dim == 3:
        rgb_img = image
    
    if not inv:
        return np.zeros((rgb_img.shape[0], rgb_img.shape[1]), dtype = np.uint8)
    else:
        return np.ones((rgb_img.shape[0], rgb_img.shape[1]), dtype = np.uint8)

## Extracting image IDs
Some Images are 5 dimensional. Also height and width of the images are non uniform. We are rescaling all images as 2000x2000.


In [ ]:
def get_image(file_name, gray = True, display = False):
    
    train_ids = ['2f6ecfcdf', 'cb2d976f4', '1e2425f28', '54f2eec69', '0486052bb', 'aaa6a05cc', 'e79de561c', '095bf7a1f']
    test_ids = ['afa5e8098', 'c68fe75ea', 'b2dc8411c', 'b9a3865fc', '26dc41664']
    
    if file_name in train_ids:
        train_or_test = "train"
    elif file_name in test_ids:
        train_or_test = "test"
    else:
        print("ID not available in dataset")
        return
    
    image = tiff.imread('/kaggle/input/hubmap-kidney-segmentation/{}/{}.tiff'.format(train_or_test, file_name))
    dim = image.ndim
    
    if dim == 5:
        rgb_img = image[0, 0, :, :, :].transpose((1, 2, 0))
        
    elif dim == 3:
        rgb_img = image
    
    if gray:
        result_unscaled = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2GRAY)
        result = cv2.resize(result_unscaled, (2000, 2000))
    else:
        result = cv2.resize(rgb_img, (2000, 2000))
    
    
    if display:
        plt.figure(figsize=(20, 20))
        plt.imshow(result, cmap='gray')
        return result
    else:
        return result

In [ ]:
image_tiff = get_image('b2dc8411c', gray=False, display=True)
    


## Coordinate shapes for Medulla and Cortex for training and test ids

In [ ]:
def get_prop_shape(file_name):
    
    train_ids = ['2f6ecfcdf', 'cb2d976f4', '1e2425f28', '54f2eec69', '0486052bb', 'aaa6a05cc', 'e79de561c', '095bf7a1f']
    test_ids = ['afa5e8098', 'c68fe75ea', 'b2dc8411c', 'b9a3865fc', '26dc41664']
    
    if file_name in train_ids:
        train_or_test = "train"
    elif file_name in test_ids:
        train_or_test = "test"
    else:
        print("ID not available in dataset")
        return
    
    def flatten(iterable):
        if len(iterable[0]) == 2:
            return iterable
        else:
            return flatten(iterable[0])
        
    anatomy = open('/kaggle/input/hubmap-kidney-segmentation/{}/{}-anatomical-structure.json'.format(train_or_test, file_name))
    json_obj = json.load(anatomy)
    
    try:
        
        result = {}
        for elem in json_obj:
            key = elem['properties']['classification']['name']
            value = np.array(flatten(elem['geometry']['coordinates']))
            if key in result.keys():
                key = str(key) + ' I'
            result[key] = value
        return result
    
    except IndexError:
        print("Error occured on {} id".format(file_name))
        
        

### Anatomical data found in training and test data set.

In [ ]:
    
print("Anatomical data found in Training data \n")

for _id in train_ids:
    print(str(_id) + '      ' + str(get_prop_shape(_id).keys()) + '\n')

print(" \nAnatomical data found in Testing data \n")

for _id in test_ids:
    print(str(_id) + '      ' + str(get_prop_shape(_id).keys()) + '\n')
    
print("\n \nThere are certainly some irregularities in testing and training data \n")                    

### Not all images in training and test images have Medulla data

In [ ]:
print("Medulla              Cortex           ID")

print("\n \n Training data \n \n")
for id_g in train_ids:
    c = get_prop_shape(id_g)
    print(str(c.get("Medulla", np.array(None)).shape) + "             " + str(c.get("Cortex", np.array(None)).shape) + "        " + id_g)
    
print(" \n \n Testing data \n \n ")

for id_g in test_ids:
    c = get_prop_shape(id_g)
    print(str(c.get("Medulla", np.array(None)).shape) + "             " + str(c.get("Cortex", np.array(None)).shape) + "        " + id_g)

## Extracting differnt anatomical features

In [ ]:
def extract_feature(file_name, gray = True, display = False):
    
    anatomy_dict = get_prop_shape(file_name)
    anato_featu = [i for i in anatomy_dict.keys()]
    
    zeros = generate_mask(file_name, False)
    
    for anatomy in anato_featu:
        coordinate = anatomy_dict[anatomy]
        mask = cv2.fillPoly(zeros, pts = [coordinate], color = 255)
    
    mask_resized = cv2.resize(mask, (2000, 2000))
    
    if not gray:
        img = get_image(file_name, gray)
        r, g, b = cv2.split(img)
        result_r = cv2.bitwise_and(r, r, mask = mask_resized)
        result_g = cv2.bitwise_and(g, g, mask = mask_resized)
        result_b = cv2.bitwise_and(b, b, mask = mask_resized)
        result = cv2.merge((result_r, result_g, result_b))
    
    else:
        img = get_image(file_name, gray)
        result = cv2.bitwise_and(img, img, mask = mask_resized)
    
    
    if display:
        plt.figure(figsize=(20, 20))
        plt.imshow(result)
        return result
    
    else:
        return result



In [ ]:
subject = extract_feature('26dc41664', display= True)

## Extracting Glomerous

In [ ]:
def get_glomerous_polygon_list(file_name):
    try:
        train_ids = ['2f6ecfcdf', 'cb2d976f4', '1e2425f28', '54f2eec69', '0486052bb', 'aaa6a05cc', 'e79de561c', '095bf7a1f']
        test_ids = ['afa5e8098', 'c68fe75ea', 'b2dc8411c', 'b9a3865fc', '26dc41664']

        if file_name in train_ids:
            train_or_test = "train"
        elif file_name in test_ids:
            train_or_test = "test"
        else:
            print("ID not available in dataset")
            return
        
        glomerulus = open('/kaggle/input/hubmap-kidney-segmentation/{}/{}.json'.format(train_or_test, file_name))
        json_obj = json.load(glomerulus)
        
        polygon_list = []
        
        for elem in json_obj:
            narr = np.array(elem['geometry']['coordinates'][0])
            polygon_list.append(narr)
        
        return polygon_list
          
    except:  
        print('Cannot load file {}'.format(file_name))
        raise

In [ ]:
def extract_glomerous(file_name, gray = True, display = False, inv = False):
    poly_list = get_glomerous_polygon_list(file_name)
    rgb = extract_feature(file_name, gray = not gray)
    gray = extract_feature(file_name, gray = gray)
    r, g, b = cv2.split(rgb)
    
    mask = generate_mask(file_name, inv)
    color = 0 if inv else 255
    
    for coordinates in poly_list:
        cv2.fillPoly(mask, pts=[coordinates], color= color)
        
    mask_resized = cv2.resize(mask, (2000, 2000))
    
    result_r = cv2.bitwise_and(r, r, mask = mask_resized)
    result_g = cv2.bitwise_and(g, g, mask = mask_resized)
    result_b = cv2.bitwise_and(b, b, mask = mask_resized)
    
    result_gray = cv2.bitwise_and(gray, gray, mask = mask_resized)
    result_rgb = cv2.merge((result_r, result_g, result_b))
    
    if gray.any():
        result = cv2.resize(result_gray, (2000, 2000))
    else:
        result = cv2.resize(result_rgb, (2000, 2000))
    
    if display:
        plt.figure(figsize=(20, 20))
        plt.imshow(result)
        return result
    
    else:
        return result

## Images of glomerous

In [ ]:
glomorous = extract_glomerous('cb2d976f4', display=True, inv = False)

## Images minus glomerous

In [ ]:
non_glomorous = extract_glomerous('cb2d976f4', display=True, inv = True)

In [ ]:
non_glomorous[0][0]

## Comparing pixel densities of glomorous and non- glomorous region

In [ ]:
fig, (ax_g, ax_ng) = plt.subplots(2, 1, figsize=(10, 10))

glomorous_flatten = glomorous.flatten()
non_glomourous_flatten = non_glomorous.flatten()

ax_g.hist(glomorous_flatten[glomorous_flatten != 0], bins=256)
ax_g.set_title('Glomorous Pixel distribution')

ax_ng.hist(non_glomourous_flatten[non_glomourous_flatten != 0], bins=256)
ax_ng.set_title('Non-Glomouros Pixel distribution')


fig.tight_layout()
plt.show()

## Slicing the image into grids for more analysis.

The above analyisis doesnt provide much information due to  very high pixel density in non-glomorous region. Hence analysing pixels in small grids of size 25x25  might make more sense.

In [ ]:
# making grid for the image and its corresponding glomorous mask
GRID_SQUARE_SIZE = 25
OVER_LAP = 8
def make_grid_img_vs_mask(file_name):
    gray = extract_feature(file_name)
    zeros = generate_mask(file_name, inv = False)
    poly_list = get_glomerous_polygon_list(file_name)
    
    for coordinates in poly_list:
        mask = cv2.fillPoly(zeros, pts=[coordinates], color= 255)
        
    mask_resized = cv2.resize(mask, (2000, 2000))
    
    gray_vs_mask = []
    for row in range(0, 2000 - GRID_SQUARE_SIZE, GRID_SQUARE_SIZE - OVER_LAP):
        for col in range(0, 2000 - GRID_SQUARE_SIZE, GRID_SQUARE_SIZE - OVER_LAP):
            gray_grid = gray[row:row + GRID_SQUARE_SIZE, col:col+GRID_SQUARE_SIZE]
            mask_grid = mask_resized[row:row + GRID_SQUARE_SIZE, col:col+GRID_SQUARE_SIZE]
            gray_vs_mask.append(np.array([gray_grid, mask_grid]))
            
    return np.array(gray_vs_mask, dtype = 'uint8')

In [ ]:
t = make_grid_img_vs_mask('cb2d976f4')
t.shape

In [ ]:
#seperate grids into grids containg glomorous and not containing.
REQ_GLOM = 10
REQ_NON_GLOB = 50
def grid_admission_glob(mask):
    
    zero_check = mask.any()>0
    
    if zero_check:
        unique, counts = np.unique(mask, return_counts=True)
        unique_mask_elem = dict(zip(unique, counts))
        no_not_zero = sum([unique_mask_elem[k] for k in unique_mask_elem if k != 0])
#         no_255 = unique_mask_elem[255]
        mask_size = np.size(mask)
        perc_255 = (no_not_zero/mask_size)*100
        req_glom_check = perc_255>REQ_GLOM
        return zero_check and req_glom_check
    else:
        return zero_check
    
def grid_admission_non_glob(gray):
    unique, counts = np.unique(gray, return_counts=True)
    unique_gray_elem = dict(zip(unique, counts))     
    if 0 in unique_gray_elem.keys():
        no_zeros = unique_gray_elem[0]
        gray_size = np.size(gray)
        perc_zero = (no_zeros/gray_size)*100
        req_glom_check = perc_zero<REQ_NON_GLOB
        return req_glom_check
    else:
        return True
    
def seperate_glom_non_glom_grid(file_name):
    glom_vs_non_glom = make_grid_img_vs_mask(file_name)
    glom = []
    non_glom = []
    
    for row in range(0, glom_vs_non_glom.shape[0]):
        gray = glom_vs_non_glom[row][0]
        mask = glom_vs_non_glom[row][1]
        
#         if mask.any()>0:
        if grid_admission_glob(mask):
            glom.append([gray, mask])
        else:
            if grid_admission_non_glob(gray):
                non_glom.append([gray, mask])
            
    return {
        "glom": np.array(glom),
        "non_glom": np.array(non_glom)
    }

In [ ]:
glom_vs_non_glom_cb2d976f4 = seperate_glom_non_glom_grid('cb2d976f4')
gloms = glom_vs_non_glom_cb2d976f4["glom"]
non_gloms = glom_vs_non_glom_cb2d976f4["non_glom"]
print('Glomerous shape - ' + str(gloms.shape))
print('\nNon-Glomerous shape - ' + str(non_gloms.shape))
print('\nThere are {} number of Glomerous grids and {} number of non-glomorous grids'.format(str(gloms.shape[0]), str(non_gloms.shape[0])))

In [ ]:
NO_OF_GRIDS = 10
START = 0
fig, ax = plt.subplots(NO_OF_GRIDS, 4, figsize=(20, 20))
for row in range(NO_OF_GRIDS):
    ax[row][0].imshow(gloms[row + START][0], 'gray')
    ax[row][0].set_title('Glomorous Grid ' + str(row +1))
    ax[row][1].imshow(gloms[row + START][1], 'gray')
    ax[row][1].set_title('Glomorous Mask Grid ' + str(row +1))
    ax[row][2].imshow(non_gloms[row + START][0], 'gray')
    ax[row][2].set_title('Non - Glomorous Grid ' + str(row +1))
    ax[row][3].imshow(non_gloms[row + START][1], 'gray')
    ax[row][3].set_title('Non - Glomorous Grid - Mask ' + str(row +1))

fig.tight_layout()
plt.show()

## Analysising Pixel histogram of grids
Lets compare pixel densitities of glomorous and non glomorous grids and finds out if there is any difference between them. We can also check if there is any similarities in glomorous grids.

In [ ]:
fig, ax = plt.subplots(NO_OF_GRIDS, 3, figsize=(20, 20))
for row in range(NO_OF_GRIDS):
    
    hist_glm_and_non = cv2.calcHist([gloms[row + START][0]],[0],None,[256],[0,256])
    ax[row][0].plot(hist_glm_and_non)
    ax[row][0].set_title('Glomorous + Non-glomorous Grid ' + str(row +1))
    
    hist_glm_only = cv2.calcHist([gloms[row + START][0]],[0],gloms[row + START][1],[256],[0,256])
    ax[row][1].plot(hist_glm_only)
    ax[row][1].set_title('Glomorous only -masked- Grid ' + str(row +1))
    
    hist_non_glm = cv2.calcHist([non_gloms[row + START][0]],[0],None,[256],[0,256])
    ax[row][2].plot(hist_non_glm)
    ax[row][2].set_title('Non - Glomorous Grid ' + str(row +1))

fig.tight_layout()
plt.show()

This is good news. In the above figure, first column is grid with glomorous and 3rd column doesnt have glomorous. It can be seen that Non - Glomorous grids have most of there pixels concentrated at 200 - 250. But in the Glomorous Grid we can see a wide components of pixel in the range from 50 to 200 with a somewhat peak at 100 - 150. Now there is a component at 200 - 250 range too in that. That is Non - Glomorous part of the grid. 